### <b>Washington State Crash Event Analysis</b>
#### --- by 

In [560]:
import pandas as pd
import numpy as np
import regex as re

import requests
import asyncio
import json as js

import time

import os

pd.set_option('display.max_rows', 9)

#### <b>1. Explorative Data Analysis</b></br>
##### <b>1.1 Introduction to Datasets</b></br>
- <b>df_data_main</b>: The original data set with an expanded column that stores the zipcode of the place where the accident occured</br>
- <b> df_metadata_crashtype</b>: Derived from the table 6 of the original dataset. This dataframe contains all the metainfo about a category of crashtype</br>

In [561]:
dir = os.path.abspath(os.path.dirname(os.getcwd())) + '/data/'

df_data_main = pd.read_csv(dir + '/output/data_with_zipcode.csv').drop(axis=1, labels='Unnamed: 0') 
df_data_main.event_zipcode = df_data_main.event_zipcode.astype(str)   # convert the default float type values into str

df_metadata_crashtype = pd.read_csv(dir + '/output/crash_type.csv').set_index(keys='type_index')

df_metadata_driver_factor = pd.read_csv(dir + '/output/driver_behavioral_factors.csv').drop(labels='Unnamed: 0', axis=1)

df_data_main.shape

C:\Users\Chaconne\AppData\Local\Temp\ipykernel_15844\3945522690.py:3: DtypeWarning: Columns (22,23,24,302,304) have mixed types. Specify dtype option on import or set low_memory=False.
  df_data_main = pd.read_csv(dir + '/output/data_with_zipcode.csv').drop(axis=1, labels='Unnamed: 0')


(4132, 306)

##### <b>1.2 Data Cleaning</b>

- The following blocks drop rows which do not have valid zipcodes (i.e. rows that do not have either a driver zipcode or an accident zipcode)

In [562]:
# drop rows which do not have an event zipcode

has_no_zipcode = df_data_main.event_zipcode.map(lambda v : v == 'nan')
df_data_main = df_data_main[df_data_main.event_zipcode != 'nan']
df_data_main.shape

(4132, 306)

In [563]:
# drop rows which do not have a person zipcode

df_data_main.dzip = df_data_main.dzip.map(
    lambda n: 0 if n ==0 else 0 if pd.isna(n) else int(n)
)
df_data_main = df_data_main[df_data_main.dzip > 10000]     # valid zip codes are all 5 digit so we filter out those with less than 5 digits
df_data_main.dzip = df_data_main.dzip.astype(str)
df_data_main.shape

(4100, 306)

- The following block cleans the <b>age</b> column. <br/>
- After observation we found that there are invalid age values such as 999 or 998, which, after cleaning, are replaced with the column mean (calculation of the mean is based on the column being filtered out of the abnomral values.)

In [564]:
age_filter = filter(lambda v: v > 0 and v < 100, df_data_main.age)
age_mean = round( np.mean(list(age_filter), dtype=float),0)

df_data_main.age = df_data_main.age.map(
    lambda v : age_mean if v < 0 or v >= 100 else v
)

#### <b> 2. Solutions
##### <b>2.1 Among drivers involved in fatal crashes, what proportion are involved in crashes in communities where they live?</b>
- <b>Visualization note</b>: a barchart / pie chart to show the proportion of non-resident and resident crash cases.

In [565]:
df_data_main['is_resident'] = df_data_main.index.map(
    lambda i: df_data_main.event_zipcode[i] == df_data_main.dzip[i]
)

df_data_main['is_driver'] = df_data_main.index.map(
    lambda i: df_data_main.loc[i, 'ptype'] == 1 and df_data_main.loc[i, 'vnumber'] == 1
)

prop = len(df_data_main[(df_data_main.is_resident == True) & (
    df_data_main.is_driver == True)]) / float(len(df_data_main[df_data_main.is_driver == True]))

print('{prop:.4f}% of the drivers are from the community where the accident occured'.format(prop = prop * 100))

24.6415% of the drivers are from the community where the accident occured


Based on our analysis, <b>24.6415%</b> of the drivers are from the community where the accident occured.

##### <b>2.2 <b>2.3 Are there differences in the <u>crash types</u> in those crashes among “residents” versus those deemed to be not “from” the area?</b>
- We will first take a look at the types of crashes among residents versus non-residents drivers. To that end, we load the metadata regarding crash types.

In [566]:
df_metadata_crashtype.head()     # this dataframe stores the meta info of the variable crashtype

,crash_type,category
type_index,,
0,No Impact,NOT CATEGORIZED
1,Drive Off Road,SINGLE DRIVER
2,Control/Traction Loss,SINGLE DRIVER
3,"Avoid Collision with Vehicle, Pedestrian, Animal",SINGLE DRIVER
4,Specifics Other,SINGLE DRIVER


In [567]:
map_crashtype_category = {  # maps a crashtype to its category
    k:v for k,v in zip(df_metadata_crashtype.index, df_metadata_crashtype.category)
}

map_crashtype_eng = {   # maps a crashtype index to its actual meaning
    k:v for k,v in zip(df_metadata_crashtype.index, df_metadata_crashtype['crash_type'])
}

df_data_main['crash_category'] = df_data_main.crashtype.map(map_crashtype_category)
df_data_main['crashtype_eng'] = df_data_main.crashtype.map(map_crashtype_eng)

In [568]:
df_data_crash = df_data_main.groupby(by=['crashtype', 'is_resident']).agg(
    case_count=pd.NamedAgg(column='par', aggfunc=len),
).reset_index()

df_temp = df_data_main.groupby('is_resident').par.agg('count')  # temporary dataframe for calculating total by is_resident
# print(df_temp)
non_resident_event_count = float(df_temp.iloc[0])
resident_event_count = float(df_temp.iloc[1])
del df_temp


def get_case_proportion(case_index:int, crash_dataframe: pd.DataFrame) -> float:
    is_resident = crash_dataframe.loc[case_index, 'is_resident']
    case_count = crash_dataframe.loc[case_index, 'case_count']
    if is_resident:
        ratio = case_count / resident_event_count
    else:
        ratio = case_count / non_resident_event_count
    return ratio


df_data_crash['case_proportion'] = df_data_crash.index.map(lambda i : get_case_proportion(i, df_data_crash))

df_data_crash

,crashtype,is_resident,case_count,case_proportion
0,0,False,18,0.005762
1,0,True,4,0.004103
2,1,False,243,0.077785
3,1,True,95,0.097436
...,...,...,...,...
100,98,False,570,0.182458
101,98,True,131,0.134359
102,99,False,1,0.000320
103,99,True,1,0.001026


- The following pivot table compares the proportion of different crash categories among the resident and non-resident groups.
- <b>Visualization note</b> There should be a paired bar chart to show the proportional differences of case categories across resident and non-resident groups

In [569]:
df_crash_pivoted = df_data_crash.pivot(index='crashtype', columns='is_resident', values=['case_proportion'])

df_crash_pivoted['diff'] = df_crash_pivoted[('case_proportion', False)] - df_crash_pivoted[('case_proportion', True)]
df_crash_pivoted.sort_values(by = 'diff', ascending = False, inplace = True)
df_crash_pivoted.drop(axis = 1, labels='diff', inplace= True)
df_crash_pivoted = df_crash_pivoted.fillna(0)

df_crash_pivoted['crashtype_eng'] = df_crash_pivoted.index.map(map_crashtype_eng)
df_crash_pivoted[:4]

case_proportion                         crashtype_eng
is_resident           False      True                            
crashtype                                                        
98                 0.182458  0.134359           Other Crash Type 
50                 0.066261  0.050256  Lateral Move (Left/Right) 
28                 0.008323  0.001026     Decelerating (Slowing) 
25                 0.016005  0.009231     Slower, Going Straight

- According to the pivot table, the largest difference of case proportion appears in the crash types<b>98, 50, 28, 25,52</b>, which are <b>Other Crash Type,  Lateral Move (Left/Right), Decelerating (Slowing), and Slower, Going Straight</b>.</br>

- We conclude that <b>there is a significantly high proportion of non-resident drivers who caused MISCELLANEOUS crash events</b>.</br>

##### <b>2.3 Are there differences in the <u>behavior factors</u> in those crashes among “residents” versus those deemed to be not “from” the area?

In [570]:
df_metadata_driver_factor = df_metadata_driver_factor.loc[
    df_metadata_driver_factor.effect_end >= df_data_main.year.min(), :
]   # filter out factors that do not take effect during the timeframe of the dataset

df_metadata_driver_factor = df_metadata_driver_factor.sort_values(by = 'factor_index', ascending=True).reset_index(drop=True)
df_metadata_driver_factor

,factor_index,driver_factor,effect_start,effect_end,factor_category
0,0,No factors identified for this driver,0,9999,default
1,4,Reaction to/Failure to take Drugs/Medication,0,9999,Physical/Mental Condition
2,6,Careless Driving,2012,9999,Physical/Mental Condition
3,8,Aggressive Driving Road Rage,2004,9999,Physical/Mental Condition
...,...,...,...,...,...
76,94,Emergency Medical Service Personnel,0,2019,Possible Distraction Inside Vehicle
77,95,Fire Personnel,0,2019,Possible Distraction Inside Vehicle
78,96,Tow Operator,0,2019,Possible Distraction Inside Vehicle
79,97,"Transportation i.e. maintenance workers, safte...",0,2019,Possible Distraction Inside Vehicle


In [571]:
def sum_driver_factor(index:int, df:pd.DataFrame):
    joint = '|'.join(df.loc[index, 'drf1':'drf4'].astype(int).astype(str))   # first conerted to int to remove trailing decimal zero
    return re.sub(pattern=r'\|0', string=joint, repl='')


df_data_main.loc[:, 'drf1':'drf4'] = df_data_main.loc[:, 'drf1':'drf4'].fillna(0)     # in this case, na suggests no factor rather than the missing of value
df_data_main['drf'] = df_data_main.index.map( lambda i:
    sum_driver_factor(i, df_data_main)
)

df_data_main.drop(axis=1, labels=['drf1','drf2','drf3','drf4'], inplace= True)  # drop the original component columns as we already acquired the summed up one

In [572]:
def get_factor_levels(factor_col):
    res = []
    for val in factor_col:
        val_split = val.split('|')
        for v in val_split:
            res.append(v)
    return sorted(np.unique(res))   # sort the levels


drf_distinct_vals = get_factor_levels(df_data_main.drf)   # stores the distinct values of drf


def factor_to_dummy(factor_col, level):
    col_len = len(factor_col)

    data_dict = {
        lv:[0] * col_len for lv in level
    }
    for i in range(col_len):
        keys = factor_col[i].split('|')
        for key in keys:
            data_dict[key][i] = 1
    return pd.DataFrame(data_dict)


df_drf_dist = factor_to_dummy(df_data_main.drf.values, drf_distinct_vals)
df_drf_dist.insert(0, 'is_resident', df_data_main.is_resident)
df_drf_dist = df_drf_dist.groupby('is_resident').agg('sum')
df_drf_dist.loc[0,:] = df_drf_dist.loc[0,:] / sum(df_drf_dist.loc[0,:])
df_drf_dist.loc[1,:] = df_drf_dist.loc[1,:] / sum(df_drf_dist.loc[1,:])
df_drf_dist = df_drf_dist.transpose()


df_drf_dist.columns = ['non-resident', 'resident']
df_drf_dist['diff'] = df_drf_dist['non-resident'] - df_drf_dist['resident']
df_drf_dist.sort_values(by='diff', inplace=True)
df_drf_dist.drop('diff', inplace = True, axis=1)
df_drf_dist.set_index(pd.Series(df_drf_dist.index, name = 'drf', dtype=int), inplace=True)
df_drf_dist[:5]

,non-resident,resident
drf,,
87,0.015625,0.022593
8,0.010723,0.014735
6,0.002145,0.005894
10,0.017157,0.020629
33,0.016238,0.019646


- The table above suggests that compared to resident drivers, non-resident drviers are much more likely to encounter accidents that result from the behavioral factor <b>87</b>, that is, <b>Skidding, Swerving, Sliding Due To Ice, Water, Snow, Slush, Sand, Dirt, Oil, or Wet Leaves on Road</b>.

- The following blocks focus on the analysis of <b>distraction factors</b>. As the distraction index 0 means not distracted, and index 96 not reported, we merge these two categories into one.

In [573]:
df_data_main.loc[:, 'distract1': 'distract6'] = df_data_main.loc[:, 'distract1': 'distract6'].fillna(0)

def sum_distract_factors(ind:int, df: pd.DataFrame):
    return re.sub(string= "|".join(df.loc[ind, 'distract1': 'distract6'].astype(int).astype(str)), pattern=r'\|0', repl ='')
    

df_data_main['distract'] = df_data_main.index.map(
    lambda i: re.sub(string=sum_distract_factors(i, df_data_main), repl='0', pattern='96')
)

df_data_main.drop(axis = 1, labels= ['distract1', 'distract2','distract3','distract4','distract5','distract6'], inplace=True)
df_data_main.distract.value_counts(ascending=False)

0       3515
92       330
93       144
12        20
        ... 
5|93       1
14         1
3|6        1
97         1
Name: distract, Length: 20, dtype: int64

In [574]:
distinct_distract = get_factor_levels(df_data_main.distract)

df_distract = factor_to_dummy(df_data_main.distract.values, distinct_distract)
df_distract.insert(0, column='is_resident', value=df_data_main.is_resident)
df_distract = df_distract.groupby('is_resident').agg('sum')
df_distract.loc[0,:] = df_distract.loc[0,:] / sum(df_distract.loc[0,:])
df_distract.loc[1,:] = df_distract.loc[1,:] / sum(df_distract.loc[1,:])
df_distract = df_distract.transpose()
df_distract.columns = ['non-resident', 'resident']
df_distract['diff'] = df_distract['non-resident'] - df_distract['resident']
df_distract.sort_values(by='diff', inplace=True)
df_distract.drop('diff', inplace = True, axis=1)
df_distract.set_index(pd.Series(df_distract.index, name = 'distract_factor', dtype=int), inplace=True)
df_distract[:5] 

,non-resident,resident
distract_factor,,
0,0.854288,0.864669
3,0.001934,0.005165
15,0.001612,0.003099
9,0.000645,0.002066
6,0.003224,0.004132


##### <b>2.4 Are there specific resident ZIP Codes that tend to produce higher-risk drivers that are involved in fatal crashes at a higher rate?</b>
- To address this question, we should first present a clear definition of <b>higher-risk drivers</b>. The following columns are considered to signify a driver's tendency towards risky driving.</br>
- <b>spdrel</b>: valued 2 if the driver was racing, valued 3 if the driver exceeded speed limit.
- <b>drugres1 through drugres12</b>: these 12 columns contain information regarding whether the driver used drugs while or before driving.
- <b>alcres</b>: this column indicates whether the driver drank alchohol.
- <b>dr_fty</b>: this column indicates if the driver failted to yield right-of-way
- <b>dr_unlic</b>: this column indicates if the driver has a valid license.

In [575]:
df_risk = df_data_main.loc[:, 'drugres1':'drugres8']
df_risk.insert(0, 'is_resident', df_data_main.is_resident)
df_risk['alcres'] = df_data_main.alcres
df_risk['dr_fty'] = df_data_main.dr_fty
df_risk['dr_unlic'] = df_data_main.dr_unlic
df_risk.head()

#list(filter(lambda s: 'drugres' in s, df_data_main.columns))

,is_resident,drugres1,drugres2,drugres3,drugtst1,drugtst2,drugtst3,alcsts,alctst,alcres,...,drugtst4,drugtst5,drugtst6,drugtst7,drugtst8,drugres4,drugres5,drugres6,drugres7,drugres8
0,False,417.0,0.0,0.0,1.0,0.0,0.0,2,1,0.056,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,False,1.0,0.0,0.0,1.0,0.0,0.0,2,1,0.190,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,False,605.0,0.0,0.0,1.0,0.0,0.0,2,1,0.140,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,False,402.0,0.0,0.0,1.0,0.0,0.0,2,1,0.051,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,False,0.0,0.0,0.0,0.0,0.0,0.0,0,0,996.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### Output Cleaned Dataset for Visualization

In [576]:
df_data_main.to_csv(dir + 'output/data_vis.csv')

##### Predictive Analysis of Risky Drivers